<a href="https://colab.research.google.com/github/arenas-franklin/PerformanceOlimpica/blob/main/preparacao_dados_olimpicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação de dados Olímpicos
---



In [11]:
import pandas as pd

In [12]:
# Carregar todos os datasets do CSV
try:
  athlete_bio_df = pd.read_csv('/content/drive/MyDrive/dataset/performace_olimpica/olympics_athlete_bio.csv')
  athlete_event_result_df = pd.read_csv('/content/drive/MyDrive/dataset/performace_olimpica/olympics_athlete_event_result.csv')
  country_df = pd.read_csv('/content/drive/MyDrive/dataset/performace_olimpica/olympics_country.csv')
  game_df = pd.read_csv('/content/drive/MyDrive/dataset/performace_olimpica/olympics_game.csv')
  game_medal_tally_df = pd.read_csv('/content/drive/MyDrive/dataset/performace_olimpica/olympics_game_medal_tally.csv')
  print("arquivos carregados com sucesso !!!")
except FileNotFoundError as e:
  print(f"Erro: O arquivo {e.filenome} não foi encontrado. Certifique-se de que todos os arquivos foram carregados corretamente.")
  exit()

/tmp/ipython-input-3861304984.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  athlete_event_result_df = pd.read_csv('/content/drive/MyDrive/dataset/performace_olimpica/olympics_athlete_event_result.csv')


arquivos carregados com sucesso !!!


## Etapa 1: Unir os dados  e preparar a base completa

In [13]:
#Unir dados de resultados de eventos com dados biográficos de atletas
combined_df = pd.merge(athlete_event_result_df, athlete_bio_df, on='athlete_id', how='left', suffixes=('_event', '_bio'))

In [14]:
# Unir com os dados os jogos para obter a data de início e o tipo de jogo
combined_df = pd.merge(combined_df, game_df[['edition_id','start_date', 'year']], on='edition_id', how='left')
combined_df = combined_df.rename(columns={'year': 'game_year'})

In [15]:
# Unir com osdados dos países para obter o nome completo do país
combined_df = pd.merge(combined_df, country_df, left_on='country_noc_event', right_on='noc', how='left')
combined_df = combined_df.rename( columns={'name':'country_full_name'})

In [16]:
# removendo colunas duplicadas ou redundantes
combined_df = combined_df.drop(columns=['country_bio', 'noc', 'country_event'],errors='ignore')

## Etapa 2: Limpeza e criado feature

In [21]:
# Extrair o tipo de jogo (summer ou Winter)
combined_df['game_type'] = combined_df['edition'].str.split(' ').str[1]

In [25]:
#Converter colunas de data para o tipo datetime
combined_df['start_date'] = pd.to_datetime(combined_df['start_date'])
combined_df['birth_date'] = pd.to_datetime(combined_df['birth_date'])

In [27]:
# Colcular a idade do atleta no época dos jogos
combined_df['age_at_game'] = (combined_df['start_date'] - combined_df['birth_date']).dt.days // 365.25

In [29]:
# Calcular  o IMC (Indice de Massa Corporal), tratando valores  ausentes
combined_df['bmi'] = combined_df['weight'] / (combined_df['height'] / 100) ** 2

In [31]:
#Unir com o total de medalhas por país em cada edição
game_medal_tally_df = game_medal_tally_df.rename(columns={
    'gold': 'country_total_gold',
    'silver': 'country_total_silver',
    'bronze': 'country_total_bronze',
    'total': 'country_total_medals',
    'country_noc': 'country_noc_event' # Renomear para corresponder
})
final_df = pd.merge(combined_df, game_medal_tally_df[['edition_id', 'country_noc_event', 'country_total_gold', 'country_total_silver', 'country_total_bronze', 'country_total_medals']], on=['edition_id', 'country_noc_event'], how='left')
final_df = final_df.rename(columns={'country_noc_event': 'country_noc'})

## Etapa 3: Salvar o arquivo final

In [33]:
output_file = 'olympics_full_performance_data.csv'
final_df.to_csv(output_file, index=False)

print(f"O arquivo '{output_file}' foi criado com sucesso.")

O arquivo 'olympics_full_performance_data.csv' foi criado com sucesso.


In [34]:
# Primeiras 5 linhas da tabela
final_df.head()

,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,position,medal,...,start_date,game_year,name_y,game_type,age_at_game,bmi,country_total_gold,country_total_silver,country_total_bronze,country_total_medals
0,1928 Winter Olympics,30,SUI,Skeleton,"Skeleton, Men",1,Willy von Eschen,98710,Did not finish,NaN,...,1928-02-11,1928,Switzerland,Winter,NaN,NaN,0.0,0.0,1.0,1.0
1,1928 Winter Olympics,30,FRA,Skeleton,"Skeleton, Men",1,"Jean, Comte de Beaumont",42118,Did not start,NaN,...,1928-02-11,1928,France,Winter,24.0,NaN,1.0,0.0,0.0,1.0
2,1928 Winter Olympics,30,FRA,Skeleton,"Skeleton, Men",1,Pierre Dormeuil,85267,Did not finish,NaN,...,1928-02-11,1928,France,Winter,40.0,22.446689,1.0,0.0,0.0,1.0
3,1928 Winter Olympics,30,GBR,Skeleton,"Skeleton, Men",1,Lord Brabazon of Tara,1202561,Did not start,NaN,...,1928-02-11,1928,Great Britain,Winter,44.0,NaN,0.0,0.0,1.0,1.0
4,1928 Winter Olympics,30,SUI,Skeleton,"Skeleton, Men",1,Alexander Berner,84063,5,NaN,...,1928-02-11,1928,Switzerland,Winter,NaN,NaN,0.0,0.0,1.0,1.0


In [35]:
# informação do DataFrame
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318019 entries, 0 to 318018
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   edition               318019 non-null  object        
 1   edition_id            318019 non-null  int64         
 2   country_noc           318019 non-null  object        
 3   sport                 318019 non-null  object        
 4   event                 318019 non-null  object        
 5   result_id             318019 non-null  int64         
 6   athlete               318019 non-null  object        
 7   athlete_id            318019 non-null  int64         
 8   position              318019 non-null  object        
 9   medal                 44996 non-null   object        
 10  is_team_sport         318019 non-null  bool          
 11  name_x                318012 non-null  object        
 12  sex                   318012 non-null  object        
 13 

In [37]:
# total de atletas ùnicos
final_df['athlete_id'].nunique()

155867